# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Data-Import-and-Preprocessing" data-toc-modified-id="Data-Import-and-Preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Import and Preprocessing</a></div><div class="lev2 toc-item"><a href="#Distances" data-toc-modified-id="Distances-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Distances</a></div><div class="lev1 toc-item"><a href="#sumprod-spatio-temporal-kernel" data-toc-modified-id="sumprod-spatio-temporal-kernel-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><code>sumprod</code> spatio-temporal kernel</a></div><div class="lev1 toc-item"><a href="#Simpler-kernel" data-toc-modified-id="Simpler-kernel-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Simpler kernel</a></div><div class="lev1 toc-item"><a href="#All-RQIso" data-toc-modified-id="All-RQIso-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>All RQIso</a></div>

Try to simplify the sum of products kernel without sacrificing marginal likelihood.

In [2]:
using TimeSeries
using DataFrames
using GaussianProcesses
using GaussianProcesses: Mean, Kernel, evaluate, metric, IsotropicData, VecF64
using GaussianProcesses: Stationary, KernelData, MatF64
import GaussianProcesses: optimize!, get_optim_target, cov, grad_slice!
import GaussianProcesses: num_params, set_params!, get_params, update_mll!, update_mll_and_dmll!
import GaussianProcesses: get_param_names, cov!, addcov!, multcov!
import Proj4
using Optim
using Distances
;

In [3]:
import PyPlot; plt=PyPlot
using LaTeXStrings
plt.rc("figure", dpi=300.0)
# plt.rc("figure", figsize=(6,4))
plt.rc("savefig", dpi=300.0)
plt.rc("text", usetex=true)
plt.rc("font", family="serif")
plt.rc("font", serif="Palatino")
;

# Data Import and Preprocessing

In [4]:
include("src/preprocessing.jl")

test_data (generic function with 1 method)

In [5]:
include("src/TempModel.jl")

TempModel

In [6]:
isdList=read_isdList()
isdList[1:5,:]

,USAF,WBAN,NAME,CTRY,STATE,ICAO,LAT,LON,ELEV,BEGIN,END,X_PRJ,Y_PRJ
1,10010,99999,JAN MAYEN(NOR-NAVY),NO,,ENJA,70.933,-8.667,9.0,1931,2015,4.5545e6,6.11344e6
2,10060,99999,EDGEOYA,NO,,,78.25,22.817,14.0,1973,2015,4.04982e6,7.5564e6
3,10070,99999,NY-ALESUND,SV,,,78.917,11.933,7.7,1973,2015,3.8678e6,7.26549e6
4,10080,99999,LONGYEAR,SV,,ENSB,78.246,15.466,26.8,1975,2015,3.99705e6,7.33669e6
5,10090,99999,KARL XII OYA,SV,,,80.65,25.0,5.0,1955,2015,3.69259e6,7.68545e6


In [7]:
isdSubset=isdList[[(usaf in (725450,725460,725480,725485)) for usaf in isdList[:USAF].values],:]
isdSubset

,USAF,WBAN,NAME,CTRY,STATE,ICAO,LAT,LON,ELEV,BEGIN,END,X_PRJ,Y_PRJ
1,725450,14990,THE EASTERN IOWA AIRPORT,US,IA,KCID,41.883,-91.717,264.6,1973,2015,1.64799e6,1.0441e6
2,725460,14933,DES MOINES INTERNATIONAL AIRPORT,US,IA,KDSM,41.534,-93.653,291.7,1973,2015,1.48723e6,1.00379e6
3,725480,94910,WATERLOO MUNICIPAL AIRPORT,US,IA,KALO,42.554,-92.401,264.6,1960,2015,1.59025e6,1.11766e6
4,725485,14940,MASON CITY MUNICIPAL ARPT,US,IA,KMCW,43.154,-93.327,373.4,1973,2015,1.51407e6,1.18374e6


In [8]:
hourly_cat=read_Stations(isdSubset)
hourly_cat[1:5,:]

,year,month,day,hour,min,seconds,temp,ts,station,ts_hours
1,2015,1,1,0,52,0,-7.8,2015-01-01T00:52:00,1,0.866667
2,2015,1,1,1,52,0,-8.3,2015-01-01T01:52:00,1,1.86667
3,2015,1,1,2,52,0,-8.3,2015-01-01T02:52:00,1,2.86667
4,2015,1,1,3,52,0,-9.4,2015-01-01T03:52:00,1,3.86667
5,2015,1,1,4,52,0,-9.4,2015-01-01T04:52:00,1,4.86667


## Distances

To get distances between stations, we can either use a function to compute distances on a sphere, or we can first project the coordinates onto a Euclidean plane, and then compute normal distances. I'll do it both ways to check they're consistent (equal up to a multiplication constant), and then use Euclidean distances for convenience.

In [9]:
# http://www.johndcook.com/blog/python_longitude_latitude/
function distance_on_unit_sphere(lat1, long1, lat2, long2)
 
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = π/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.
         
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
     
    cosangle = (sin(phi1)*sin(phi2)*cos(theta1 - theta2) +
           cos(phi1)*cos(phi2))
    arc = acos( cosangle )
 
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc
end

distance_on_unit_sphere (generic function with 1 method)

In [10]:
numstations = nrow(isdSubset)
pairwiseSphere = zeros(numstations, numstations)
for i in 1:numstations
    for j in 1:i
        if i==j
            continue
        end
        station1 = isdSubset[i,:]
        station2 = isdSubset[j,:]
        lat1= get(station1[1,:LAT])
        lon1 = get(station1[1,:LON])
        lat2 = get(station2[1,:LAT])
        lon2 = get(station2[1,:LON])
        pairwiseSphere[i,j] = distance_on_unit_sphere(lat1, lon1, lat2, lon2)
        pairwiseSphere[j,i] = pairwiseSphere[i,j]
    end
end
pairwiseSphere

4×4 Array{Float64,2}:
 0.0        0.0259496  0.0146736  0.0303475
 0.0259496  0.0        0.024088   0.0285853
 0.0146736  0.024088   0.0        0.0158124
 0.0303475  0.0285853  0.0158124  0.0      

In [11]:
pairwiseEuclid=pairwise(Euclidean(), Matrix(isdSubset[[:X_PRJ,:Y_PRJ]])')

4×4 Array{Float64,2}:
      0.0        165736.0        93510.4        1.93474e5
 165736.0             0.0            1.53559e5  1.81942e5
  93510.4             1.53559e5      0.0        1.00846e5
      1.93474e5       1.81942e5      1.00846e5  0.0      

Ratio of the two distance matrices: close enough to a constant!

In [12]:
pairwiseEuclid ./ pairwiseSphere

4×4 Array{Float64,2}:
 NaN            6.38684e6    6.37271e6    6.37527e6
   6.38684e6  NaN            6.37493e6    6.36489e6
   6.37271e6    6.37493e6  NaN            6.37765e6
   6.37527e6    6.36489e6    6.37765e6  NaN        

# `sumprod` spatio-temporal kernel

In [13]:
k1 = fix(Periodic(0.0,0.0,log(24.0)), :lp)
k2 = RQIso(0.0,0.0,0.0)
k3 = SEIso(0.0,0.0)
k4 = RQIso(0.0,0.0,0.0)
k5 = RQIso(0.0,0.0,0.0)
k6 = SEIso(0.0,0.0)

ksp1 = SEIso(log(2*10^5), log(1.0))
ksp2 = SEIso(log(2*10^5), log(1.0))
ksp3 = SEIso(log(2*10^5), log(1.0))
ksp4 = SEIso(log(2*10^5), log(1.0))
ksp5 = SEIso(log(2*10^5), log(1.0))
ksp6 = SEIso(log(2*10^5), log(1.0))
k_means = SEIso(log(10^4), log(10.0))

k_spatiotemporal_1 = Masked(k1, [1]) * Masked(ksp1, [2,3]) +
                 Masked(k2, [1]) * Masked(ksp2, [2,3]) +
                 Masked(k3, [1]) * Masked(ksp3, [2,3]) +
                 Masked(k4, [1]) * Masked(ksp4, [2,3]) +
                 Masked(k5, [1]) * Masked(ksp5, [2,3]) +
                 Masked(k6, [1]) * Masked(ksp6, [2,3]) +
                 fix(Masked(k_means, [2,3]))
hyp = [-1.68206,-0.179317,0.945821,13.5116,0.0501475,0.866468,0.758593,-0.984024,11.0867,-0.38583,-1.44126,-1.13345,9.20607,0.0421904,2.12626,1.24119,-0.15271,15.081,0.129167,3.68457,0.701431,3.00982,14.0459,-1.5127,7.70676,-5.30466,12.2061,-6.18869e-6]
set_params!(k_spatiotemporal_1, hyp[2:end])
logNoise=hyp[1]

-1.68206

In [14]:
begin
    k_spatiotemporal = k_spatiotemporal_1
    chunks=GP[]
    chunk_width=24*10
    tstart=0.0
    tend=tstart+chunk_width
    nobsv=0
    while tstart < get(maximum(hourly_cat[:ts_hours]))
        in_chunk=(tstart .<= hourly_cat[:ts_hours].values) & (hourly_cat[:ts_hours].values .< tend)
        hourly_chunk = hourly_cat[in_chunk,:]
        nobsv_chunk = sum(in_chunk)
        nobsv += nobsv_chunk

        chunk_X_PRJ = isdSubset[:X_PRJ].values[hourly_chunk[:station].values]
        chunk_Y_PRJ = isdSubset[:Y_PRJ].values[hourly_chunk[:station].values]
        chunk_X = [hourly_chunk[:ts_hours].values chunk_X_PRJ chunk_Y_PRJ]

        y = hourly_chunk[:temp].values
        chunk = GP(chunk_X', y, MeanConst(mean(y)), k_spatiotemporal, logNoise)
        push!(chunks, chunk)

        tstart=tend
        tend+=chunk_width
    end
    reals = TempModel.GPRealisations(chunks)
    update_mll_and_dmll!(reals, mean=false)
    @printf(" mLL: %.5f\n", reals.mLL)
    println("dmLL: ", reals.dmLL)
end

 mLL: -46183.86261
dmLL: [-10.1656,4.76353,-0.0412465,12.6709,-0.0412465,30.7611,-24.8592,-19.845,8.76812,-24.8592,-0.406135,-27.4537,-1.4777e-15,-27.4537,-12.7652,-0.977735,-57.2385,-6.39481,-0.977735,-0.0381473,3.15524,0.00941418,-0.229537,3.15524,-1.32648e-6,-2.9701e-5,6.72606e-7,-2.9701e-5]


In [15]:
print("k₁: Periodic \n=================\n")
@printf("σ: %5.3f\n", √k1.kern.σ2)
@printf("l: %5.3f\n", √k1.kern.ℓ2)
@printf("p: %5.0f hours\n", k1.kern.p)
print("> spatial decay:\n")
@printf("l: %5.3f\n", √ksp1.ℓ2)
print("\nk₂: RQIso \n=================\n")
@printf("σ: %5.3f\n", √ k2.σ2)
@printf("l: %5.3f hours\n", √ k2.ℓ2)
@printf("α: %5.3f\n", k2.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp2.σ2)
@printf("l: %5.3f\n", √ksp2.ℓ2)
print("\nk₃: SEIso \n=================\n")
@printf("σ: %5.3f\n", √k3.σ2)
@printf("l: %5.3f hours\n", √k3.ℓ2)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp3.σ2)
@printf("l: %5.3f\n", √ksp3.ℓ2)
print("\nk₄: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k4.σ2)
@printf("l: %5.3f days\n", √k4.ℓ2 / 24)
@printf("α: %5.3f\n",  k4.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp4.σ2)
@printf("l: %5.3f\n", √ksp4.ℓ2)
print("\nk₅: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k5.σ2)
@printf("l: %5.3f days\n", √k5.ℓ2 / 24)
@printf("α: %5.3f\n",  k5.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp5.σ2)
@printf("l: %5.3f\n", √ksp5.ℓ2)
print("\nk₆ SEIso \n=================\n")
@printf("σ: %5.3f\n", √k6.σ2)
@printf("l: %5.3f days\n", √k6.ℓ2 / 24)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp6.σ2)
@printf("l: %5.3f\n", √ksp6.ℓ2)
print("\n=================\n")
@printf("σy: %5.3f\n", exp(logNoise))

k₁: Periodic 
σ: 2.575
l: 0.836
p:    24 hours
> spatial decay:
l: 737926.865

k₂: RQIso 
σ: 2.135
l: 2.378 hours
α: 0.374
> spatial decay:
l: 65296.910

k₃: SEIso 
σ: 0.322
l: 0.237 hours
> spatial decay:
l: 9957.387

k₄: RQIso 
σ: 3.460
l: 0.349 days
α: 0.858
> spatial decay:
l: 3544827.299

k₅: RQIso 
σ: 2.017
l: 1.659 days
α: 20.284
> spatial decay:
l: 1259090.257

k₆ SEIso 
σ: 0.005
l: 92.639 days
> spatial decay:
l: 200005.471

σy: 0.186


# Simpler kernel

In [16]:
k_spatiotemporal_2 = Masked(k1, [1]) * Masked(ksp1, [2,3]) +
                 Masked(k2, [1]) * Masked(ksp2, [2,3]) +
                 Masked(k3, [1]) * Masked(ksp3, [2,3]) +
                 Masked(k4, [1]) * Masked(ksp4, [2,3]) +
                 fix(Masked(k_means, [2,3]))

Type: GaussianProcesses.SumKernel
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.FixedKern}, Params: [-0.179317,0.945821]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [13.5116,0.0501475]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.RQIso}, Params: [0.866468,0.758593,-0.984024]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [11.0867,-0.38583]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [-1.44126,-1.13345]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [9.20607,0.0421904]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.RQIso}, Params: [2.12626,1.24119,-0.15271]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [15.081,0.129167]
  Type: GaussianProcesses.FixedKern, Params: Float64[]


In [17]:
begin
    k_spatiotemporal = k_spatiotemporal_2
    chunks=GP[]
    chunk_width=24*10
    tstart=0.0
    tend=tstart+chunk_width
    nobsv=0
    while tstart < get(maximum(hourly_cat[:ts_hours]))
        in_chunk=(tstart .<= hourly_cat[:ts_hours].values) & (hourly_cat[:ts_hours].values .< tend)
        hourly_chunk = hourly_cat[in_chunk,:]
        nobsv_chunk = sum(in_chunk)
        nobsv += nobsv_chunk

        chunk_X_PRJ = isdSubset[:X_PRJ].values[hourly_chunk[:station].values]
        chunk_Y_PRJ = isdSubset[:Y_PRJ].values[hourly_chunk[:station].values]
        chunk_X = [hourly_chunk[:ts_hours].values chunk_X_PRJ chunk_Y_PRJ]

        y = hourly_chunk[:temp].values
        chunk = GP(chunk_X', y, MeanConst(mean(y)), k_spatiotemporal, logNoise)
        push!(chunks, chunk)

        tstart=tend
        tend+=chunk_width
    end
    reals = TempModel.GPRealisations(chunks)
    update_mll_and_dmll!(reals, mean=false)
    @printf(" mLL: %.5f\n", reals.mLL)
    println("dmLL: ", reals.dmLL)
end

 mLL: -46185.49457
dmLL: [-10.1645,4.76959,-0.0506269,12.6698,-0.0506269,31.1047,-23.3183,-20.0028,8.87426,-23.3183,-0.40377,-27.441,-1.47773e-15,-27.441,-10.162,7.7745,-57.881,-6.41317,7.7745]


# All RQIso

In [18]:
ll, lσ = get_params(k3)
k3b = RQIso(ll,lσ,2.0)

Type: GaussianProcesses.RQIso, Params: [-1.44126,-1.13345,2.0]


In [19]:
k_spatiotemporal_3 = Masked(k1, [1]) * Masked(ksp1, [2,3]) +
                 Masked(k2, [1])  * Masked(ksp2, [2,3]) +
                 Masked(k3b, [1]) * Masked(ksp3, [2,3]) +
                 Masked(k4, [1])  * Masked(ksp4, [2,3]) +
                 fix(Masked(k_means, [2,3]))

Type: GaussianProcesses.SumKernel
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.FixedKern}, Params: [-0.179317,0.945821]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [13.5116,0.0501475]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.RQIso}, Params: [0.866468,0.758593,-0.984024]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [11.0867,-0.38583]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.RQIso}, Params: [-1.44126,-1.13345,2.0]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [9.20607,0.0421904]
  Type: GaussianProcesses.ProdKernel
    Type: GaussianProcesses.Masked{GaussianProcesses.RQIso}, Params: [2.12626,1.24119,-0.15271]
    Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [15.081,0.129167]
  Type: GaussianProcesses.FixedKern, Params: Float64[]


In [20]:
begin
    global opt_out
    k_spatiotemporal = k_spatiotemporal_3
    chunks=GP[]
    chunk_width=24*10
    tstart=0.0
    tend=tstart+chunk_width
    nobsv=0
    while tstart < get(maximum(hourly_cat[:ts_hours]))
        in_chunk=(tstart .<= hourly_cat[:ts_hours].values) & (hourly_cat[:ts_hours].values .< tend)
        hourly_chunk = hourly_cat[in_chunk,:]
        nobsv_chunk = sum(in_chunk)
        nobsv += nobsv_chunk

        chunk_X_PRJ = isdSubset[:X_PRJ].values[hourly_chunk[:station].values]
        chunk_Y_PRJ = isdSubset[:Y_PRJ].values[hourly_chunk[:station].values]
        chunk_X = [hourly_chunk[:ts_hours].values chunk_X_PRJ chunk_Y_PRJ]

        y = hourly_chunk[:temp].values
        chunk = GP(chunk_X', y, MeanConst(mean(y)), k_spatiotemporal, logNoise)
        push!(chunks, chunk)

        tstart=tend
        tend+=chunk_width
    end
    reals = TempModel.GPRealisations(chunks)
    update_mll_and_dmll!(reals, mean=false)
    @printf(" mLL: %.5f\n", reals.mLL)
    println("dmLL: ", reals.dmLL)
    @time opt_out=TempModel.optimize_NLopt(reals, mean=false, x_tol=1e-5, f_tol=1e-10)
    print("optimization -->")
    @printf(" mLL: %.5f\n", reals.mLL)
    println("dmLL: ", reals.dmLL)
end

 mLL: -46168.78799
dmLL: [-21.8953,4.62217,-0.0414086,12.7326,-0.0414086,35.3355,-34.3684,-22.9631,14.4427,-34.3684,-35.8925,133.353,-13.1731,-1.48982e-15,133.353,-9.49386,7.56329,-57.7423,-6.40906,7.56329]
9070.302024 seconds (91.01 G allocations: 2.210 TB, 7.93% gc time)
optimization --> mLL: -45944.62514
dmLL: [-0.000205554,-0.000292355,-1.42258e-5,-9.40713e-5,-1.42258e-5,-0.00308438,0.00353499,0.000350118,-0.00141016,0.00353499,7.0301e-5,0.00217324,-1.27168,-6.35772e-16,0.00217324,0.00014801,-0.000181258,5.80736e-5,0.000403622,-0.000181258]


In [21]:
begin
    global opt_out
    k_spatiotemporal = k_spatiotemporal_3
    chunks=GP[]
    chunk_width=24*10
    tstart=0.0
    tend=tstart+chunk_width
    nobsv=0
    while tstart < get(maximum(hourly_cat[:ts_hours]))
        in_chunk=(tstart .<= hourly_cat[:ts_hours].values) & (hourly_cat[:ts_hours].values .< tend)
        hourly_chunk = hourly_cat[in_chunk,:]
        nobsv_chunk = sum(in_chunk)
        nobsv += nobsv_chunk

        chunk_X_PRJ = isdSubset[:X_PRJ].values[hourly_chunk[:station].values]
        chunk_Y_PRJ = isdSubset[:Y_PRJ].values[hourly_chunk[:station].values]
        chunk_X = [hourly_chunk[:ts_hours].values chunk_X_PRJ chunk_Y_PRJ]

        y = hourly_chunk[:temp].values
        chunk = GP(chunk_X', y, MeanConst(mean(y)), k_spatiotemporal, logNoise)
        push!(chunks, chunk)

        tstart=tend
        tend+=chunk_width
    end
    reals = TempModel.GPRealisations(chunks)
    update_mll_and_dmll!(reals, mean=false)
    @printf(" mLL: %.5f\n", reals.mLL)
    println("dmLL: ", reals.dmLL)
    @time opt_out=TempModel.optimize_NLopt(reals, mean=false, x_tol=1e-5, f_tol=1e-10)
    print("optimization -->")
    @printf(" mLL: %.5f\n", reals.mLL)
    println("dmLL: ", reals.dmLL)
end

 mLL: -45955.78939
dmLL: [-520.112,5.84728,-1.22125,0.196695,-1.22125,74.3353,-43.435,6.31325,6.1487,-43.435,185.501,-344.184,-46.3423,-6.00122e-16,-344.184,3.1481,-1.28973,0.755154,0.499024,-1.28973]
4022.710679 seconds (41.11 G allocations: 1.000 TB, 7.90% gc time)
optimization --> mLL: -45944.57887
dmLL: [0.000689847,0.00020083,-0.00244532,-0.015515,-0.00244532,-0.00318563,0.00162387,0.00174717,-0.000917766,0.00162387,-0.000911702,0.00269821,0.000462429,-6.36516e-16,0.00269821,-0.00273785,0.000395533,0.0117028,-0.0012609,0.000395533]


In [25]:
opt_out

(45944.578866171665,[-1.72527,-0.210656,0.950124,13.574,0.0544504,0.653978,0.538881,0.125985,10.9932,-0.605542,-1.2208,-0.946048,-1.0721,9.20607,0.229593,2.18355,1.29463,-1.14171,12.8224,0.182608],:FTOL_REACHED,35)

In [27]:
print(opt_out[2])

[-1.72527,-0.210656,0.950124,13.574,0.0544504,0.653978,0.538881,0.125985,10.9932,-0.605542,-1.2208,-0.946048,-1.0721,9.20607,0.229593,2.18355,1.29463,-1.14171,12.8224,0.182608]

In [22]:
print(get_params(k_spatiotemporal_3))

[-0.210656,0.950124,13.574,0.0544504,0.653978,0.538881,0.125985,10.9932,-0.605542,-1.2208,-0.946048,-1.0721,9.20607,0.229593,2.18355,1.29463,-1.14171,12.8224,0.182608]

In [24]:
print("k₁: Periodic \n=================\n")
@printf("σ: %5.3f\n", √k1.kern.σ2)
@printf("l: %5.3f\n", √k1.kern.ℓ2)
@printf("p: %5.0f hours\n", k1.kern.p)
print("> spatial decay:\n")
@printf("l: %5.3f\n", √ksp1.ℓ2)
print("\nk₂: RQIso \n=================\n")
@printf("σ: %5.3f\n", √ k2.σ2)
@printf("l: %5.3f hours\n", √ k2.ℓ2)
@printf("α: %5.3f\n", k2.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp2.σ2)
@printf("l: %5.3f\n", √ksp2.ℓ2)
print("\nk₃ᵇ: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k3b.σ2)
@printf("l: %5.3f hours\n", √k3b.ℓ2)
@printf("α: %5.3f\n", k3b.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp3.σ2)
@printf("l: %5.3f\n", √ksp3.ℓ2)
print("\nk₄: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k4.σ2)
@printf("l: %5.3f days\n", √k4.ℓ2 / 24)
@printf("α: %5.3f\n",  k4.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp4.σ2)
@printf("l: %5.3f\n", √ksp4.ℓ2)
print("\nk₅: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k5.σ2)
@printf("l: %5.3f days\n", √k5.ℓ2 / 24)
@printf("α: %5.3f\n",  k5.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp5.σ2)
@printf("l: %5.3f\n", √ksp5.ℓ2)
print("\nk₆ SEIso \n=================\n")
@printf("σ: %5.3f\n", √k6.σ2)
@printf("l: %5.3f days\n", √k6.ℓ2 / 24)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp6.σ2)
@printf("l: %5.3f\n", √ksp6.ℓ2)
print("\n=================\n")
@printf("σy: %5.3f\n", exp(logNoise))

k₁: Periodic 
σ: 2.586
l: 0.810
p:    24 hours
> spatial decay:
l: 785403.011

k₂: RQIso 
σ: 1.714
l: 1.923 hours
α: 1.134
> spatial decay:
l: 59470.046

k₃ᵇ: RQIso 
σ: 0.388
l: 0.295 hours
α: 0.342
> spatial decay:
l: 9957.387

k₄: RQIso 
σ: 3.650
l: 0.370 days
α: 0.319
> spatial decay:
l: 370417.719

k₅: RQIso 
σ: 2.017
l: 1.659 days
α: 20.284
> spatial decay:
l: 1259090.257

k₆ SEIso 
σ: 0.005
l: 92.639 days
> spatial decay:
l: 200005.471

σy: 0.186
